In [5]:
import pandas as pd

# Load the datasets (replace 'path_to_file' with actual file paths if running locally)
og_data = pd.read_csv('og-unclean-data.csv')
spatial_data = pd.read_csv('spatial-unclean-data.csv')
temporal_data = pd.read_csv('temporal-unclean-data.csv')

# Step 1: Drop irrelevant columns
og_data_cleaned = og_data.drop(columns=['Unnamed: 13', 'Unnamed: 14'])
spatial_data_cleaned = spatial_data.drop(columns=['Unnamed: 3'])
temporal_data_cleaned = temporal_data.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

# Step 2: Handle missing values
# Forward fill for consistency
og_data_cleaned.fillna(method='ffill', inplace=True)

# Drop rows where MPA or FisheryConsumption is missing in spatial and temporal datasets
spatial_data_cleaned.dropna(subset=['MPA', 'Fishery'], inplace=True)
temporal_data_cleaned.dropna(subset=['MPA', 'FisheryConsumption'], inplace=True)

# The cleaned datasets
cleaned_og_data = og_data_cleaned
cleaned_spatial_data = spatial_data_cleaned
cleaned_temporal_data = temporal_data_cleaned

# Save the cleaned datasets to CSV files

# Paths for saving cleaned data
cleaned_og_data_path = 'cleaned_og_data.csv'
cleaned_spatial_data_path = 'cleaned_spatial_data.csv'
cleaned_temporal_data_path = 'cleaned_temporal_data.csv'

# Save the cleaned data
cleaned_og_data.to_csv(cleaned_og_data_path, index=False)
cleaned_spatial_data.to_csv(cleaned_spatial_data_path, index=False)
cleaned_temporal_data.to_csv(cleaned_temporal_data_path, index=False)

cleaned_og_data_path, cleaned_spatial_data_path, cleaned_temporal_data_path


# Output cleaned data to confirm the result
cleaned_og_data.head(), cleaned_spatial_data.head(), cleaned_temporal_data.head()


/var/folders/g9/s_fx5qk126515rbzxjm9x6ww0000gn/T/ipykernel_36532/1630005942.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  og_data_cleaned.fillna(method='ffill', inplace=True)


(                  Country Name        2017        2018        2019  \
 0                        Aruba      149.50      150.00      164.50   
 1  Africa Eastern and Southern  3897511.90  3883753.45  3821770.86   
 2                  Afghanistan     9000.00    10000.00    10729.00   
 3   Africa Western and Central  3706566.44  3642143.61  3667626.70   
 4                       Angola   532914.00   444858.00   401696.00   
 
          2020        2021        2022    2017.1    2018.1    2019.1    2020.1  \
 0      161.50      166.50      173.50  0.000105  0.000105  0.000105  0.000105   
 1  3805060.99  4097350.48  3725334.12  4.945690  4.945687  5.235880  5.036564   
 2    11444.00    12907.00    13150.00  4.945690  4.945687  5.235880  5.036564   
 3  3523165.88  3701624.19  3894655.40  4.945690  4.945687  5.235880  5.036564   
 4   376515.00   531522.00   471672.00  0.004930  0.004930  0.004930  0.004930   
 
      2021.1    2022.1  
 0  0.000105  0.000105  
 1  5.040512  0.000105  
 2 